In [3]:
import numpy as np
from IPython.display import clear_output
import time
import random
from termcolor import colored
from tqdm import tqdm
import pickle
from multiprocess import Pool
from functools import partial
import random
# https://www.youtube.com/watch?v=UXW2yZndl7U

# Bitboards

### bitboard class, add piece, check win

In [13]:
class Bitboard:
    def __init__(self):
        self.player1 = 0  # Bitboard for Player 1
        self.player2 = 0  # Bitboard for Player 2
        self.height = [0] * 7  # Tracks the next free row in each column
        self.rows = 6
        self.cols = 7

    def print_board(self):
        """Prints the board in a human-readable format."""
        board = [[' ' for _ in range(self.cols)] for _ in range(self.rows)]
        for col in range(self.cols):
            for row in range(self.rows):
                index = col * self.rows + row
                if self.player1 & (1 << index):
                    board[row][col] = colored('X', 'red') # Player 1
                elif self.player2 & (1 << index):
                    board[row][col] = colored('O', 'yellow')  # Player 2
        for row in reversed(board):  # Print top row first
            print('|' + '|'.join(row) + '|')
        print('-' * (2 * self.cols + 1))

In [5]:
# replaces the update_board fcn
def add_piece(bitboard, column, player):
    """
    Add a piece to the given column for the specified player.
    :param bitboard: Bitboard instance.
    :param column: The column (0-6) to play in.
    :param player: 1 (Player 1) or 2 (Player 2).
    """
    if column < 0 or column >= bitboard.cols or bitboard.height[column] >= bitboard.rows:
        raise ValueError("Invalid column or column is full.")
    index = column * bitboard.rows + bitboard.height[column]
    if player == 1:
        bitboard.player1 |= (1 << index)  # Set bit in Player 1's bitboard
    else:
        bitboard.player2 |= (1 << index)  # Set bit in Player 2's bitboard
    bitboard.height[column] += 1  # Update the next free row in the column


In [6]:
# replaces the check_for_win fcn
def check_winner(bitboard, player):
    """
    Check if the given player has won.
    :param bitboard: Bitboard instance.
    :param player: 1 (Player 1) or 2 (Player 2).
    :return: True if the player has won, False otherwise.
    """
    board = bitboard.player1 if player == 1 else bitboard.player2
    rows = bitboard.rows
    cols = bitboard.cols

    # Check horizontal
    horizontal = board & (board >> 1)
    if horizontal & (horizontal >> 2):
        return True

    # Check vertical
    vertical = board & (board >> rows)
    if vertical & (vertical >> (2 * rows)):
        return True

    # Check diagonal (top-left to bottom-right)
    diagonal1 = board & (board >> (rows + 1))
    if diagonal1 & (diagonal1 >> (2 * (rows + 1))):
        return True

    # Check anti-diagonal (top-right to bottom-left)
    diagonal2 = board & (board >> (rows - 1))
    if diagonal2 & (diagonal2 >> (2 * (rows - 1))):
        return True

    return False

### mcts

In [7]:
# updates find_legal fcn
def find_legal(bitboard):
    """
    Find all legal moves (columns not full).
    :param bitboard: Bitboard instance.
    :return: List of legal column indices.
    """
    return [col for col in range(bitboard.cols) if bitboard.height[col] < bitboard.rows]


In [8]:
def look_for_win(bitboard, player):
    """
    Check if the current player has an immediate winning move.
    :param bitboard: The current board state (Bitboard instance).
    :param player: The current player (1 = Player 1, 2 = Player 2).
    :return: The column index of a winning move, or -1 if no such move exists.
    """
    legal_moves = find_legal(bitboard)  # Get all legal columns
    for col in legal_moves:
        # Simulate placing a piece in the column
        temp_board = Bitboard()
        temp_board.player1, temp_board.player2 = bitboard.player1, bitboard.player2
        temp_board.height = bitboard.height[:]
        add_piece(temp_board, col, player)

        # Check if this move results in a win
        if check_winner(temp_board, player):
            return col  # Found a winning move
    return -1  # No winning move found


In [9]:
def find_all_nonlosers(bitboard, player):
    """
    Find all legal moves that do not allow the opponent to win immediately.
    :param bitboard: The current board state (Bitboard instance).
    :param player: The current player (1 = Player 1, 2 = Player 2).
    :return: A list of column indices for "safe" moves.
    """
    opponent = 3 - player  # Opponent player (1 ↔ 2)
    legal_moves = find_legal(bitboard)  # Get all legal columns
    safe_moves = []

    for col in legal_moves:
        # Simulate the current player's move
        temp_board = Bitboard()
        temp_board.player1, temp_board.player2 = bitboard.player1, bitboard.player2
        temp_board.height = bitboard.height[:]
        add_piece(temp_board, col, player)

        # Check if the opponent has an immediate winning move after this
        opponent_wins = False
        for opp_col in find_legal(temp_board):
            # Simulate the opponent's move
            temp_temp_board = Bitboard()
            temp_temp_board.player1, temp_temp_board.player2 = temp_board.player1, temp_board.player2
            temp_temp_board.height = temp_board.height[:]
            add_piece(temp_temp_board, opp_col, opponent)

            if check_winner(temp_temp_board, opponent):
                opponent_wins = True
                break

        # If no immediate win for the opponent, add the move to safe moves
        if not opponent_wins:
            safe_moves.append(col)

    return safe_moves


In [10]:
# updates rollout fcn
def rollout(bitboard, next_player):
    """
    Perform a random rollout from the given board state.
    :param bitboard: Bitboard instance.
    :param next_player: 1 (Player 1) or 2 (Player 2).
    :return: The winner ('X', 'O', or 'tie').
    """
    current_player = next_player
    while True:
        legal = find_legal(bitboard)
        if not legal:
            return 'tie'  # No legal moves = tie
        move = random.choice(legal)
        add_piece(bitboard, move, current_player)
        if check_winner(bitboard, current_player):
            return 'X' if current_player == 1 else 'O'
        current_player = 3 - current_player  # Switch between 1 and 2


In [11]:
def mcts(bitboard, color0, nsteps):
    """
    Perform Monte Carlo Tree Search (MCTS) to determine the best move.
    :param bitboard: Bitboard instance representing the current board state.
    :param color0: Starting player ('X' = Player 1, 'O' = Player 2).
    :param nsteps: Number of MCTS simulations to run.
    :return: The best column to play.
    """
    # Initialize the MCTS dictionary with the current board state.
    # Key: (player1_bitboard, player2_bitboard)
    # val: [total visits, total score]
    mcts_dict = {(bitboard.player1, bitboard.player2): [0, 0]}
    # Step 1: Immediate checks
    # get all legal moves
    legal_moves = find_legal(bitboard)

    # check for an immediate winning move
    win_move = look_for_win(bitboard, 1 if color0 == 'X' else 2)
    if win_move != -1:
        return win_move  # Return the winning move immediately

    # check for moves that let the opponent win immediately
    safe_moves = find_all_nonlosers(bitboard, 1 if color0 == 'X' else 2)
    if safe_moves:  # If there are safe moves, use them
        legal_moves = safe_moves
    # else:
    #     print("Warning: No safe moves found; using all legal moves.")

    # If no safe moves exist, move forward with all legal moves

    # Step 2: Initialize MCTS simulation
    for _ in range(nsteps):  # Perform `nsteps` simulations
        path = []  # Path of visited states
        # Copy the initial board
        temp_board = Bitboard()
        temp_board.player1, temp_board.player2 = bitboard.player1, bitboard.player2
        temp_board.height = bitboard.height[:]
        current_player = 1 if color0 == 'X' else 2 # current player (1 = Player 1, 2 = Player 2)
        winner = None

        # Selection & Expansion Phase
        while True:
            state_key = (temp_board.player1, temp_board.player2) # Represent the current board state as a tuple
            path.append(state_key) # Add this state to the path

            # If the state is not in the MCTS dictionary, initialize it
            if state_key not in mcts_dict:
                mcts_dict[state_key] = [0, 0]  # [total visits, total score]
                win_move = look_for_win(temp_board, current_player)
                if win_move != -1:
                    add_piece(temp_board, win_move, current_player)
                    winner = 'X' if current_player == 1 else 'O'
                    break
                break

            # UCB1 calculation to select the best move
            legal_moves = find_legal(temp_board)
            safe_moves = find_all_nonlosers(temp_board, current_player)
            if safe_moves:
                legal_moves = safe_moves
            if not legal_moves:
                winner = 'tie'  # No moves = tie
                break

            ucb1_scores = []
            parent_visits = mcts_dict[state_key][0]
            for col in legal_moves:
                # Generate next state
                next_board = Bitboard()
                next_board.player1, next_board.player2 = temp_board.player1, temp_board.player2
                next_board.height = temp_board.height[:]
                add_piece(next_board, col, current_player)
                next_key = (next_board.player1, next_board.player2)

                # UCB1 calculation
                if next_key not in mcts_dict: # if total visits == 0
                    ucb1_scores.append(float('inf'))  # Prioritize unexplored states
                else:
                    visits, score = mcts_dict[next_key]
                    # exploitation term + exploration term
                    # exploitation term: how favorable this move has been so far
                    # exploration term: encourages exploration of moves visited less oftern relative to parent state
                    ucb1 = (score / visits) + 2 * np.sqrt(np.log(parent_visits) / (1 + visits))
                    ucb1_scores.append(ucb1)

            # Choose the best move based on UCB1
            # will choose the highest ucb1 score
            # unvisited states will be inf and will be chosen
            best_move_idx = np.argmax(ucb1_scores)
            chosen_move = legal_moves[best_move_idx]
            add_piece(temp_board, chosen_move, current_player)

            # check if the move wins the game
            if check_winner(temp_board, current_player):
                winner = 'X' if current_player == 1 else 'O'
                break

            # game is not over so switch to the other player
            current_player = 3 - current_player # switch between Player 1 and Player 2

        # Rollout Phase
        # if the while loop is exited due to new state discovery
        if winner is None:
            winner = rollout(temp_board, current_player)

        # Backpropagation Phase
        for i, state in enumerate(reversed(path)):
            mcts_dict[state][0] += 1  # Increment visits
            if winner == 'tie':
                continue  # No score adjustment for ties
            if (winner == 'X' and (i % 2 == 0)) or (winner == 'O' and (i % 2 == 1)):
                mcts_dict[state][1] += 1  # Favorable for the player who made the move
            else:
                mcts_dict[state][1] -= 1  # Unfavorable for the player who made the move

    # Step 3: Choose the Best Move
    best_score = -float('inf')
    best_col = -1
    for col in find_legal(bitboard):
        temp_board = Bitboard()
        temp_board.player1, temp_board.player2 = bitboard.player1, bitboard.player2
        temp_board.height = bitboard.height[:]
        add_piece(temp_board, col, 1 if color0 == 'X' else 2)
        state_key = (temp_board.player1, temp_board.player2)
        if state_key in mcts_dict:
            visits, score = mcts_dict[state_key]
            avg_score = score / visits if visits > 0 else -float('inf')
            if avg_score > best_score:
                best_score = avg_score
                best_col = col

    return best_col


### testing

In [12]:
# Simulate a single game with MCTS playing against itself
def play_single_game():
    bitboard = Bitboard()  # Initialize an empty board
    current_player = 'X'  # Player 1 starts
    nsteps = 2000  # Number of simulations per decision

    print("Starting Connect 4 Game:")
    bitboard.print_board()  # Display the initial empty board

    while True:
        print(f"\n{current_player}'s turn...")

        # Run MCTS to determine the best move for the current player
        best_move = mcts(bitboard, current_player, nsteps)

        # Simulate the move
        add_piece(bitboard, best_move, 1 if current_player == 'X' else 2)

        # Display the board after the move
        print(f"played column {best_move+1} of 7")
        bitboard.print_board()

        # Check if the game is over
        if check_winner(bitboard, 1):
            print("Player 1 (X) wins!")
            break
        if check_winner(bitboard, 2):
            print("Player 2 (O) wins!")
            break
        if not find_legal(bitboard):  # No legal moves = tie
            print("The game is a tie!")
            break

        # Switch players
        current_player = 'O' if current_player == 'X' else 'X'


In [13]:
play_single_game()

Starting Connect 4 Game:
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------

X's turn...
played column 5 of 7
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | |X| | |
---------------

O's turn...
played column 1 of 7
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|O| | | |X| | |
---------------

X's turn...
played column 6 of 7
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|O| | | |X|X| |
---------------

O's turn...
played column 3 of 7
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|O| |O| |X|X| |
---------------

X's turn...
played column 2 of 7
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|O|X|O| |X|X| |
---------------

O's turn...
played column 2 of 7
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| |O| | | | | |
|O|X|O| |X|X| |
---

### dataset creation

#### initial creation, not parallel

In [14]:
def generate_dataset(num_games, nsteps, save_path):
    """
    Generate a dataset of Connect 4 board states and MCTS-determined best moves.
    :param num_games: Number of games to simulate.
    :param nsteps: Number of MCTS simulations per move.
    :param save_path: Path to save the dataset.
    """
    dataset = []  # Store the board states and moves

    for game in tqdm(range(num_games), desc="Simulating games"):
        # Initialize an empty board
        bitboard = Bitboard()
        current_player = 'X'  # Player X starts

        while True:
            # Check if the game is over
            if check_winner(bitboard, 1):
                break  # Player 1 wins
            if check_winner(bitboard, 2):
                break  # Player 2 wins
            if not find_legal(bitboard):  # No legal moves = tie
                break
            # Run MCTS to determine the best move
            best_move = mcts(bitboard, current_player, nsteps)
            # Record the board state and best move
            dataset.append({
                'state': (bitboard.player1, bitboard.player2),
                'best_move': best_move
            })
            # Simulate the chosen move
            add_piece(bitboard, best_move, 1 if current_player == 'X' else 2)
            # Switch players
            current_player = 'O' if current_player == 'X' else 'X'

    # Save the dataset to a file
    save_dataset(dataset, save_path)
    print(f"Dataset generation complete! Saved {len(dataset)} states to {save_path}")

def save_dataset(dataset, save_path):
    """
    Save the dataset to a file.
    :param dataset: The dataset to save.
    :param save_path: Path to save the dataset.
    """
    with open(save_path, 'wb') as f:
        pickle.dump(dataset, f)



In [15]:
# num_games = 3000  # Number of games to simulate
# nsteps = 1500      # Number of MCTS simulations per move
# save_path = "connect4_dataset.pkl"  # Path to save the dataset

# # Generate the dataset
# generate_dataset(num_games, nsteps, save_path)


#### parallelized creation

In [42]:
def generate_single_game(nsteps):
    """
    Simulate a single Connect 4 game and update the progress counter.
    :param nsteps: Number of MCTS simulations per move.
    :return: A list of states and best moves for the game.
    """
    dataset = []  # Store states for this game
    bitboard = Bitboard()  # Initialize an empty board
    current_player = 'X'

    # Simulate the game
    while True:
        if check_winner(bitboard, 1):
            break  # Player 1 wins
        if check_winner(bitboard, 2):
            break  # Player 2 wins
        if not find_legal(bitboard):  # No legal moves = tie
            break
        # Use MCTS to determine the best move
        best_move = mcts(bitboard, current_player, nsteps)
        # Record the state and best move
        dataset.append({
            'state': (bitboard.player1, bitboard.player2),
            'best_move': best_move
        })
        # Apply the move
        add_piece(bitboard, best_move, 1 if current_player == 'X' else 2)
        # Switch players
        current_player = 'O' if current_player == 'X' else 'X'
    return dataset


In [41]:
def worker_single_game(game_id, nsteps):
    """
    A top-level function that calls single-game simulation.
    Must be defined at the module level so it can be pickled on Windows.
    """
    # return generate_single_game(nsteps)
    print("Worker started")
    result = generate_single_game(nsteps)
    print("Worker finished")
    return result


In [40]:
def parallel_generate_dataset(num_games, nsteps, save_path, processes):
    """
    Parallel dataset generation using a top level worker fcn with progress tracking using tqdm.
    :param num_games: Number of games to simulate.
    :param nsteps: Number of MCTS simulations per move.
    :param save_path: Path to save the generated dataset.
    :param processes: Number of parallel processes to use.
    """
    worker = partial(worker_single_game, nsteps=nsteps)
    dataset = []

    # Initialize a multiprocessing pool
    with Pool(processes=processes) as pool:
        print("pooling")
        # Use tqdm to track progress
        with tqdm(total=num_games, desc="Generating games") as pbar:
            print("tqdming")
            # run_game = partial(generate_single_game, nsteps)
            for game_result in pool.imap_unordered(worker, range(num_games)):
                print("extending")
                dataset.extend(game_result)
                pbar.update(1)

    # Save the dataset to a file
    save_dataset(dataset, save_path)
    print(f"Dataset generation complete! Saved {len(dataset)} states to {save_path}")


In [38]:
# test_game = generate_single_game(nsteps=10)
# print(test_game)
worker_single_game(_, 10)


Worker started
Worker finished


[{'state': (0, 0), 'best_move': 0},
 {'state': (1, 0), 'best_move': 0},
 {'state': (1, 2), 'best_move': 0},
 {'state': (5, 2), 'best_move': 1},
 {'state': (5, 66), 'best_move': 1},
 {'state': (133, 66), 'best_move': 1},
 {'state': (133, 322), 'best_move': 1},
 {'state': (645, 322), 'best_move': 0},
 {'state': (645, 330), 'best_move': 1},
 {'state': (1669, 330), 'best_move': 5},
 {'state': (1669, 1073742154), 'best_move': 0},
 {'state': (1685, 1073742154), 'best_move': 0},
 {'state': (1685, 1073742186), 'best_move': 2},
 {'state': (5781, 1073742186), 'best_move': 1},
 {'state': (5781, 1073744234), 'best_move': 3},
 {'state': (267925, 1073744234), 'best_move': 3},
 {'state': (267925, 1074268522), 'best_move': 4},
 {'state': (17045141, 1074268522), 'best_move': 2},
 {'state': (17045141, 1074276714), 'best_move': 2},
 {'state': (17061525, 1074276714), 'best_move': 4},
 {'state': (17061525, 1107831146), 'best_move': 5},
 {'state': (2164545173, 1107831146), 'best_move': 2},
 {'state': (21645

In [39]:
if __name__ == "__main__":
    parallel_generate_dataset(
        num_games=10,  # Small number of games for testing
        nsteps=10,    # Fewer simulations for quicker execution
        save_path="test_dataset.pkl",
        processes=8    # Use the same process count as intended for the full run
    )


pooling


Generating games:   0%|          | 0/10 [00:00<?, ?it/s]

tqdming


NameError: name 'generate_single_game' is not defined

In [ ]:
import os
print("Number of CPU cores:", os.cpu_count())

Number of CPU cores: 16


In [ ]:
num_games = 10000  # Number of games to simulate
nsteps = 1500      # Number of MCTS simulations per move
save_path = "connect4_dataset_parallel.pkl"  # Path to save the dataset

# Generate the dataset
parallel_generate_dataset_with_progress(num_games, nsteps, save_path, processes=12)


### dataset inspection

In [7]:
# Load the dataset
save_path = "connect4_rand.pkl"
with open(save_path, "rb") as f:
    dataset = pickle.load(f)

print(f"Loaded dataset from {save_path} with {len(dataset)} entries.")

Loaded dataset from connect4_rand.pkl with 141164 entries.


In [8]:
count = 0
for entry in dataset:
    if entry == 'state':
        count += 1
print(count)

0


In [9]:
malformed_entries = [entry for entry in dataset if not isinstance(entry, dict) or 'state' not in entry or 'best_move' not in entry]
print(f"Number of malformed entries: {len(malformed_entries)}")
print(f"Examples of malformed entries: {malformed_entries[:10]}")
valid_entries = [entry for entry in dataset if isinstance(entry, dict) and isinstance(entry.get('state'), tuple) and isinstance(entry.get('best_move'), int)]
print(f"Number of valid entries: {len(valid_entries)}")


Number of malformed entries: 0
Examples of malformed entries: []
Number of valid entries: 141164


In [11]:
# Display the first 15 entries
for i, entry in enumerate(dataset[100000:100010]):
    print(f"Entry {i + 1}: {entry}")


Entry 1: {'state': (68719481923, 17564544), 'best_move': 4}
Entry 2: {'state': (68753036355, 17564544), 'best_move': 3}
Entry 3: {'state': (68753036355, 18613120), 'best_move': 3}
Entry 4: {'state': (68755133507, 22807424), 'best_move': 3}
Entry 5: {'state': (68763522115, 1096549248), 'best_move': 1}
Entry 6: {'state': (68763524163, 1096549248), 'best_move': 2}
Entry 7: {'state': (68763524163, 1096557440), 'best_move': 2}
Entry 8: {'state': (68763540547, 1096557440), 'best_move': 0}
Entry 9: {'state': (68763540547, 1096557444), 'best_move': 0}
Entry 10: {'state': (206202494027, 1096557460), 'best_move': 4}


In [ ]:
print(dataset[:-1])

In [108]:
dataset_weighted = dataset.copy()
for entry in dataset:
    # print(entry)
    ideal_move = entry['best_move']
    if ideal_move == 2 or ideal_move == 4:
        dataset_weighted.append(entry)
    elif ideal_move == 3:
        dataset_weighted.append(entry)
        dataset_weighted.append(entry)

In [109]:
from collections import Counter

# Group entries by `state` and determine the most common `best_move`
def filter_duplicates(dataset):
    state_to_moves = {}
    
    # Group all best_moves for each state
    for entry in dataset:
        state = entry['state']
        best_move = entry['best_move']
        if state not in state_to_moves:
            state_to_moves[state] = []
        state_to_moves[state].append(best_move)
    
    # Create a new dataset with the most common best_move for each state
    filtered_dataset = []
    for state, moves in state_to_moves.items():
        most_common_move = Counter(moves).most_common(1)[0][0]  # Get the most common best_move
        filtered_dataset.append({'state': state, 'best_move': most_common_move})
    
    return filtered_dataset

# Apply the filtering function to your dataset
filtered_dataset = filter_duplicates(dataset_weighted)
filtered_original = filter_duplicates(dataset)
# Verify the result
print(f"Original dataset size: {len(dataset)}")
print(f"Filtered dataset size: {len(filtered_dataset)}")
print(f"Filtered dataset size: {len(filtered_original)}")



Original dataset size: 443661
Filtered dataset size: 334489
Filtered dataset size: 334489


In [12]:
count = {}
for i, entry in enumerate(dataset):
    if entry['state'] == (0, 0):
        best_move = entry['best_move']
        count[best_move] = count.get(best_move, 0) + 1
print(count)


{2: 746, 3: 1977, 1: 400, 4: 724, 5: 378, 0: 182, 6: 148}


In [93]:
# Check for duplicate states
unique_states = set(entry['state'] for entry in filtered_dataset)
print(f"Unique states: {len(unique_states)}")


Unique states: 334489


In [15]:
invalid_moves = [entry for entry in dataset if entry['best_move'] < 0 or entry['best_move'] > 6]
print(f"Number of invalid moves: {len(invalid_moves)}")


Number of invalid moves: 0


In [112]:
from collections import Counter

move_distribution_fil_wei = Counter(entry['best_move'] for entry in filtered_dataset)
move_distribution_orig = Counter(entry['best_move'] for entry in dataset)
move_distribution_fil_orig = Counter(entry['best_move'] for entry in filtered_original)

moves_wei = move_distribution_orig - move_distribution_fil_wei
moves_unwei = move_distribution_orig - move_distribution_fil_orig

print("Move distribution original:", move_distribution_orig)
print("Move distribution weighted:", move_distribution_fil_wei)
print("Move distribution unweight:", move_distribution_fil_orig)

print("weighted differences:", moves_wei)
print("unweight differences:", moves_unwei)

moves_delta = move_distribution_fil_wei - move_distribution_fil_orig

print("change from weighting:", moves_delta)



Move distribution original: Counter({3: 68381, 2: 67214, 4: 64284, 1: 64034, 0: 61173, 5: 60131, 6: 58444})
Move distribution weighted: Counter({3: 53906, 2: 51711, 4: 49360, 1: 47377, 0: 45042, 5: 44086, 6: 43007})
Move distribution unweight: Counter({3: 51472, 2: 50879, 4: 48532, 1: 48411, 0: 46041, 5: 45124, 6: 44030})
weighted differences: Counter({1: 16657, 0: 16131, 5: 16045, 2: 15503, 6: 15437, 4: 14924, 3: 14475})
unweight differences: Counter({3: 16909, 2: 16335, 4: 15752, 1: 15623, 0: 15132, 5: 15007, 6: 14414})
change from weighting: Counter({3: 2434, 2: 832, 4: 828})


In [8]:
import random

# Choose a random entry
bitboard_ran = Bitboard()
random_entry = random.choice(dataset)
print(f"Random Board State: {random_entry}")
bitboard_ran.player1, bitboard_ran.player2 = random_entry['state'][0], random_entry['state'][1]
bitboard_ran.print_board()

NameError: name 'Bitboard' is not defined

In [9]:
print(f"Total entries: {len(dataset)}")
print(f"Unique states: {len(unique_states)}")
print(f"Move distribution: {move_distribution}")


Total entries: 443661
Unique states: 334489
Move distribution: Counter({3: 68381, 2: 67214, 4: 64284, 1: 64034, 0: 61173, 5: 60131, 6: 58444})
